In [25]:
%matplotlib inline
import os
import pandas as pd
import seaborn as sns
import ydata_profiling
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, StandardScaler, Normalizer

extract_path = './AI_Project_Data'

# Import des données

In [26]:
csv_employee_survey_data = os.path.join(extract_path, 'employee_survey_data.csv')
csv_manager_survey_data = os.path.join(extract_path, 'manager_survey_data.csv')
csv_general_data = os.path.join(extract_path, 'general_data.csv')
csv_out_time = os.path.join(extract_path, 'out_time.csv')
csv_in_time = os.path.join(extract_path, 'in_time.csv')

employee_survey_df = pd.read_csv(csv_employee_survey_data)
manager_survey_df = pd.read_csv(csv_manager_survey_data)
general_df = pd.read_csv(csv_general_data)
out_time_df = pd.read_csv(csv_out_time)
in_time_df = pd.read_csv(csv_in_time)

# init empty dataframe
work_info = pd.DataFrame()

# Analyse des données

## Overviews

In [27]:
# employee_survey_report = ydata_profiling.ProfileReport(employee_survey_data, title='Employee Survey Data')
# employee_survey_report.to_notebook_iframe()

# manager_survey_report = ydata_profiling.ProfileReport(manager_survey_data, title='Manager Survey Data')
# manager_survey_report.to_notebook_iframe()

# general_data_report = ydata_profiling.ProfileReport(general_data, title='General Data')
# general_data_report.to_notebook_iframe()

# out_time_report = ydata_profiling.ProfileReport(out_time_data, title='Out Time')
# out_time_report.to_notebook_iframe()

# in_time_report = ydata_profiling.ProfileReport(in_time_data, title='In Time')
# in_time_report.to_notebook_iframe()

# Nettoyage des données

## Valeurs dupliquées

In [28]:
work_info.duplicated().sum()

np.int64(0)

## Valeurs constantes

Lorsqu’une colonne ne contient qu’une seule valeur unique sur l’ensemble des échantillons, elle n’apporte aucune variance ni information discriminante pour un modèle prédictif. La suppression de ces colonnes permet de réduire la dimensionnalité des données, d’accélérer les calculs et de simplifier la modélisation.

``Appui scientifique :`` La réduction de la dimensionnalité est une étape clé dans le prétraitement des données, comme le souligne l’étude de Mélanie Glasson-Cicognani et André Berchtold. (2010) Dans l'[Imputation des données manquantes: Comparaison de
différentes approches](https://inria.hal.science/file/index/docid/494698/filename/p37.pdf), qui met en avant son impact positif sur la performance des modèles.



In [29]:
# Remove columns with only one unique value
def remove_constant_columns(df):
    return df.loc[:, df.nunique() > 1]

### TODO : Trouver un article/étude expliquant ce choix

## Valeurs manquantes

Les valeurs manquantes sont remplacées par la modalité la plus fréquente (également appelée "mode"). Cette stratégie est adaptée lorsque la colonne présente une forte proportion de valeurs similaires.

``Appui scientifique :`` Selon Acuna & Rodriguez (2004), l’imputation par la valeur la plus fréquente minimise les perturbations dans les distributions des données qualitatives.

[Imputation de mode combler les lacunes grace a l imputation de mode une approche de tendance centrale](https://fastercapital.com/fr/contenu/Imputation-de-mode---combler-les-lacunes-grace-a-l-imputation-de-mode---une-approche-de-tendance-centrale.html)

In [30]:
# Fill missing values with the most frequent value
def fill_categorical_na(df):
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    if len(categorical_cols) == 0:
        return df

    categorical_imputer = SimpleImputer(strategy='most_frequent')
    
    df.loc[:, categorical_cols] = categorical_imputer.fit_transform(df[categorical_cols])
    
    return df


Les valeurs manquantes sont remplacées par la médiane. Contrairement à la moyenne, la médiane est moins sensible aux valeurs aberrantes (outliers) et préserve la robustesse de l'analyse statistique.

``Appui scientifique :`` Rubin (1987) explique que l’imputation par la médiane est une stratégie robuste, particulièrement efficace dans des distributions asymétriques.

[Imputation de données manquantes]('https://www.math.univ-toulouse.fr/~besse/Wikistat/pdf/st-m-app-idm.pdf')

In [31]:
# Fill missing values with the median rounded to the nearest integer 
def fill_numeric_na(df):
    for col in df.select_dtypes(include=['number', 'float64']).columns:
        median_value = df[col].median()
        df.loc[:, col] = df[col].fillna(round(median_value))
    return df

Lorsque les années totales de travail sont manquantes, utiliser les années passées dans l’entreprise comme proxie est une stratégie logique.

In [32]:
# Fill TotalWorkingYears with YearsAtCompany if missing
def fill_total_working_years(df):
    if 'TotalWorkingYears' in df.columns and 'YearsAtCompany' in df.columns:
        df.loc[:, 'TotalWorkingYears'] = df['TotalWorkingYears'].fillna(df['YearsAtCompany'])
    return df

## Type des valeurs

### Simplification des types

In [33]:
# Fill YearsSinceLastPromotion with YearsAtCompany if missing
def simplify_numeric_columns(df):
    df = df.apply(lambda col: col.astype(int) if col.dtype == 'float64' and col.dropna().mod(1).eq(0).all() else col)
    return df

### Conversion des type object en valeur numérique

La conversion des colonnes binaires en booléens permet une manipulation plus simple des données et une meilleure compatibilité avec les modèles d’apprentissage automatique.

``Appui scientifique :`` Dans les modèles supervisés, la représentation booléenne des données binaires est plus intuitive et réduit les problèmes d’encodage, comme le montre l’étude de Pedregosa et al. (2011).

In [34]:
# Transfrom categorical columns with two unique values into binary columns
def transform_attrition_to_bool(df):
    if 'Attrition' in df.columns:
        df.loc[:, 'Attrition'] = df['Attrition'].map({'Yes': True, 'No': False})
    return df

In [35]:
# Transform categorical columns with one-hot encoding
def encode_categorical_columns(df):
    categorical_cols = df.select_dtypes(include=['object']).columns
    if len(categorical_cols) == 0:
        return df
    
    if 'Attrition' in categorical_cols:
        categorical_cols = categorical_cols.drop('Attrition')

    encoder = OneHotEncoder(dtype=int, sparse_output=False)
    encoded_array = encoder.fit_transform(df[categorical_cols])

    encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(categorical_cols), index=df.index)
    
    df = df.drop(columns=categorical_cols).join(encoded_df)
    return df

## in_time et out_time

In [36]:
def reverse_and_merge(df):
    in_time_melted = in_time_df.melt(id_vars=['Unnamed: 0'], var_name='date', value_name='arrival_time')
    out_time_melted = out_time_df.melt(id_vars=['Unnamed: 0'], var_name='date', value_name='departure_time')

    # Renommer la colonne EmployeeID
    in_time_melted.rename(columns={'Unnamed: 0': 'EmployeeID'}, inplace=True)
    out_time_melted.rename(columns={'Unnamed: 0': 'EmployeeID'}, inplace=True)

    # Fusionner les deux DataFrames sur 'id' et 'date'
    merged_clock_in = pd.merge(in_time_melted, out_time_melted, on=['EmployeeID', 'date'], how='outer')
    return merged_clock_in

In [37]:
def generate_work_column(df):
    df['arrival_time'] = pd.to_datetime(df['arrival_time'])
    df['departure_time'] = pd.to_datetime(df['departure_time'])

    # Calculer le temps travaillé (différence entre départ et arrivée)
    df['worked_time'] = df['departure_time'] - df['arrival_time']

    # Convertir en heures pour avoir un format lisible
    df['worked_hours'] = df['worked_time'].dt.total_seconds() / 3600

    # Trier par id et date
    df.sort_values(by=['EmployeeID', 'date'], inplace=True)

    # Moyenne de la durée de travail par jour pour chaque employé
    mean_worked_hours = df.groupby('EmployeeID')['worked_hours'].mean()

    # Nombre total d'heures travaillées par employé
    total_worked_hours = df.groupby('EmployeeID')['worked_hours'].sum()

    # Nombre de jours ou le worked_hours est non nul
    worked_days = df[df['worked_hours'] > 0].groupby('EmployeeID')['worked_hours'].count()

    # Faire un data frame avec ces 3 informations pour chaque employé avec l'EmployeeID comme index
    work_info = pd.concat([mean_worked_hours, total_worked_hours, worked_days], axis=1)
    work_info.columns = ['mean_worked_hours', 'total_worked_hours', 'worked_days']
    return work_info


## Pipeline

In [38]:
default_preprocessor = Pipeline([
    ('remove_constants', FunctionTransformer(remove_constant_columns, validate=False)),
    ('fill_categorical_na', FunctionTransformer(fill_categorical_na, validate=False)),
    ('fill_numeric_na', FunctionTransformer(fill_numeric_na, validate=False)),
    ('encode_categorical_columns', FunctionTransformer(encode_categorical_columns, validate=False)),
    ('simplify_numeric_columns', FunctionTransformer(simplify_numeric_columns, validate=False)),
])

employee_survey_preprocessor = Pipeline([
    ('default_preprocessor', default_preprocessor),
])

manager_survey_preprocessor = Pipeline([
    ('default_preprocessor', default_preprocessor),
])

general_preprocessor = Pipeline([
    ('fill_total_working_years', FunctionTransformer(fill_total_working_years, validate=False)),
    ('transform_attrition_to_bool', FunctionTransformer(transform_attrition_to_bool, validate=False)),
    ('default_preprocessor', default_preprocessor),
])

work_info_preprocessor = Pipeline([
    ('reverse_and_merge', FunctionTransformer(reverse_and_merge, validate=False)),
    ('generate_work_column', FunctionTransformer(generate_work_column, validate=False)),
])

# Nettoyage des données
employee_survey_data = employee_survey_preprocessor.fit_transform(employee_survey_df)
manager_survey_data = manager_survey_preprocessor.fit_transform(manager_survey_df)
general_data = general_preprocessor.fit_transform(general_df)
work_info = work_info_preprocessor.fit_transform(work_info)

## Merge

In [54]:
full_data = general_data.merge(employee_survey_data, on='EmployeeID').merge(manager_survey_data, on='EmployeeID').merge(work_info, on='EmployeeID')
full_data.head()

,Age,Attrition,DistanceFromHome,Education,EmployeeID,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,...,MaritalStatus_Married,MaritalStatus_Single,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,mean_worked_hours,total_worked_hours,worked_days
0,51,False,6,2,1,1,131160,1,11,0,...,1,0,3,4,2,3,3,7.373651,1710.686944,232
1,31,True,10,1,2,1,41890,0,23,1,...,0,1,3,2,4,2,4,7.718969,1821.676667,236
2,32,False,17,4,3,4,193280,1,15,3,...,1,0,2,2,1,3,3,7.013240,1697.204167,242
3,38,False,2,5,4,3,83210,3,11,3,...,1,0,4,4,3,2,3,7.193678,1690.514444,235
4,32,False,10,1,5,1,23420,4,12,2,...,0,1,4,1,3,3,3,8.006175,1961.512778,245


# Analyse

In [53]:
full_data_report = ydata_profiling.ProfileReport(full_data, title='Full Data')
# full_data_report.to_notebook_iframe()

## Distribution Statistiques

### chi2

In [76]:
from scipy.stats import chi2_contingency,fisher_exact

for(col) in full_data.columns:
    contingency_table = pd.crosstab(full_data[col], full_data['Attrition'])
    
    # Calcul du chi²
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    test = fisher_exact(contingency_table)
    
    # Affichage des résultats
    #print(f"Statistique du chi² : {chi2}")
    #print(f"Degrés de liberté : {dof}")
    #print("Tableau des fréquences attendues :\n", expected)
    
    # Interprétation
    #if p < 0.0000001:
    if p < 0:
        print(p)
        print(col)

1.1886524349105674e-51
Age
0.0
Attrition
2.098176387479194e-10
DistanceFromHome
8.232120330919514e-265
MonthlyIncome
2.398606294143102e-12
NumCompaniesWorked
5.271119397132432e-55
TotalWorkingYears
1.0190499837424213e-40
YearsAtCompany
2.7530023754864433e-08
YearsSinceLastPromotion
4.373856774654096e-38
YearsWithCurrManager
3.101187049602304e-14
BusinessTravel_Travel_Frequently
3.053515416647591e-09
EducationField_Human Resources
7.65783248665915e-09
MaritalStatus_Divorced
1.961886973701806e-09
MaritalStatus_Married
3.8748991469768417e-31
MaritalStatus_Single
5.78051169604359e-14
EnvironmentSatisfaction
3.258771407020784e-11
JobSatisfaction
9.768882772950701e-11
WorkLifeBalance


# Export

In [42]:
# Export des données nettoyées
full_data.to_csv(os.path.join(extract_path, 'cleaned_data.csv'), index=False)

**Questions**

Identification des valeurs dupliquées ? 
Pourquoi on ne delete pas les lignes avec des valeurs manquantes ? A justifier  
Identification des distributions statistiques des données ?  Chi2 - log normale - normale - ... ??
Gestion des différences de nombre de valeurs pour chaque catégorie ? 
tests de dépendance inter-variables ? 
feature engineering ? feature selection module sklearn
drop Id after merge ? not useful for the model

VarianceThreshold -> éliminer les variables qui varient peu / pas (ça peu dégrader la performance du modèle dans certains cas) : SEUIL DE VARIANCE à DEFINIR NOUS MEME
chi2 - anova : a tester 
SelectKBest -> identifie les variables de X dont y dépend le plus. 
SelectFromModel -> Train a model and extract the most important coeffs. (ce que j'ai fait avec les random forests avant le check 1)
RFE -> Recursive Feature Elimination
RFECV -> RFE avec cross validation

In [43]:
from sklearn.feature_selection import SelectKBest, VarianceThreshold, chi2, SelectFromModel, RFE, RFECV
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
from collections import defaultdict

def compare_feature_selection_methods(X, y, k_features):
    # Dictionnaire pour stocker les features sélectionnées par chaque méthode
    selected_features = defaultdict(list)
    
    # VarianceThreshold
    selector = VarianceThreshold(threshold=0)
    selector.fit_transform(X)
    selected_features['VarianceThreshold'] = list(X.columns[selector.get_support()])
    
    # SelectKBest
    selector = SelectKBest(chi2, k=k_features)
    selector.fit_transform(X, y)
    selected_features['SelectKBest'] = list(X.columns[selector.get_support()])
    
    # SelectFromModel
    selector = SelectFromModel(RandomForestClassifier(n_estimators=100))
    selector.fit_transform(X, y)
    selected_features['SelectFromModel'] = list(X.columns[selector.get_support()])
    
    # RFE
    selector = RFE(RandomForestClassifier(n_estimators=100), 
                  n_features_to_select=k_features, 
                  step=1)
    selector.fit_transform(X, y)
    selected_features['RFE'] = list(X.columns[selector.get_support()])
    
    # RFECV
    selector = RFECV(RandomForestClassifier(n_estimators=100), 
                    min_features_to_select=k_features,
                    step=1, 
                    cv=5)
    selector.fit_transform(X, y)
    selected_features['RFECV'] = list(X.columns[selector.support_])
    
    # Créer un DataFrame avec le nombre maximal de caractéristiques
    max_features = max(len(features) for features in selected_features.values())
    
    # Remplir avec None pour avoir des colonnes de même longueur
    for method in selected_features:
        selected_features[method].extend([None] * (max_features - len(selected_features[method])))
    
    # Créer le DataFrame final
    features_table = pd.DataFrame(selected_features)
    
    # Ajouter des statistiques sur les features sélectionnées
    print("\nNombre de caractéristiques sélectionnées par méthode:")
    for method in selected_features:
        count = sum(1 for x in selected_features[method] if x is not None)
        print(f"{method}: {count} features")
    
    return features_table

# Utilisation
X = full_data.drop(columns=['EmployeeID', 'Attrition'])
y = full_data['Attrition'].astype(int)

features_table = compare_feature_selection_methods(X, y, k_features=44)
print("\nTableau des caractéristiques sélectionnées:")
features_table


Nombre de caractéristiques sélectionnées par méthode:
VarianceThreshold: 47 features
SelectKBest: 44 features
SelectFromModel: 17 features
RFE: 44 features
RFECV: 47 features

Tableau des caractéristiques sélectionnées:


,VarianceThreshold,SelectKBest,SelectFromModel,RFE,RFECV
0,Age,Age,Age,Age,Age
1,DistanceFromHome,DistanceFromHome,DistanceFromHome,DistanceFromHome,DistanceFromHome
2,Education,Education,Education,Education,Education
3,JobLevel,JobLevel,MonthlyIncome,JobLevel,JobLevel
4,MonthlyIncome,MonthlyIncome,NumCompaniesWorked,MonthlyIncome,MonthlyIncome
5,NumCompaniesWorked,NumCompaniesWorked,PercentSalaryHike,NumCompaniesWorked,NumCompaniesWorked
6,PercentSalaryHike,PercentSalaryHike,TotalWorkingYears,PercentSalaryHike,PercentSalaryHike
7,StockOptionLevel,StockOptionLevel,TrainingTimesLastYear,StockOptionLevel,StockOptionLevel
8,TotalWorkingYears,TotalWorkingYears,YearsAtCompany,TotalWorkingYears,TotalWorkingYears
9,TrainingTimesLastYear,TrainingTimesLastYear,YearsSinceLastPromotion,TrainingTimesLastYear,TrainingTimesLastYear
